In [7]:
import os
import os.path as osp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

## Latex: dim = 128, lr = 0.01

In [8]:
biases = [False, True]
lrs = [0.01, 0.005, 0.001]
hidden_dims = [128, 256, 512]
models = ['transe', 'distmult', 'complex', 'rotate']
datasets = ['cn15k', 'nl27k', 'ppi5k']
confi_score_functions = ['logi', 'rect']
fc_layers = ['l1', 'l3', 'none']
val_results = {}
test_results = {}
result_base_path = '/home/mou/Projects/UKGE-FL/results'
for bias in biases:
    for dataset in datasets:
        # print(f'Processing {dataset}')
        print("\\begin{landscape}")
        print("\\begin{table*}[]")
        print("\\centering")
        print("\\caption{Validation ",f"{dataset.upper()}", f"bias = {bias}","}")
        print("\\begin{tabular}{lllllllllll}")
        print("\\toprule")
        print("Model & Act & FC & MSE & MSE(l) & MAE & MAE(l) & nDCG(ll) & nDCG(l) & nDCG(e) & nDCG(el)\\\\")
        for model in models:
            print("\\midrule")
            for confi in confi_score_functions:
                for fc in fc_layers:
                    result_path = osp.join(result_base_path, f'unc_{dataset}_{model}_confi_{confi}_fc_{fc}_bias_{bias}_dim_128', f'lr_0.01_wd_0.0_topk_True')
                    val_df = pd.read_csv(osp.join(result_path, 'val_metrics.csv'))
                    min_mse = val_df['MSE'].min()
                    min_mse_epoch = val_df[val_df['MSE'] == min_mse]['Epoch'].values[0]
                    min_mae = val_df['MAE'].min()
                    min_mae_epoch = val_df[val_df['MAE'] == min_mae]['Epoch'].values[0]
                    max_ndcg_lin = val_df['nDCG_lin'].max()
                    max_ndcg_lin_epoch = val_df[val_df['nDCG_lin'] == max_ndcg_lin]['Epoch'].values[0]
                    max_ndcg_exp = val_df['nDCG_exp'].max()
                    max_ndcg_exp_epoch = val_df[val_df['nDCG_exp'] == max_ndcg_exp]['Epoch'].values[0]
                    last_mse = val_df['MSE'].iloc[-1]
                    last_mae = val_df['MAE'].iloc[-1]
                    last_ndcg_lin = val_df['nDCG_lin'].iloc[-1]
                    last_ndcg_exp = val_df['nDCG_exp'].iloc[-1]
                    # if dataset not in val_results:
                    #     val_results[dataset] = {}
                    # if model not in val_results[dataset]:
                    #     val_results[dataset][model] = {}
                    # if confi not in val_results[dataset][model]:
                    #     val_results[dataset][model][confi] = {}
                    # val_results[dataset][model][confi][fc] = {
                    #     'min_mse': min_mse,
                    #     'min_mse_epoch': min_mse_epoch,
                    #     'min_mae': min_mae,
                    #     'min_mae_epoch': min_mae_epoch,
                    #     'max_ndcg_lin': max_ndcg_lin,
                    #     'max_ndcg_lin_epoch': max_ndcg_lin_epoch,
                    #     'max_ndcg_exp': max_ndcg_exp,
                    #     'max_ndcg_exp_epoch': max_ndcg_exp_epoch,
                    #     'last_mse': last_mse,
                    #     'last_mae': last_mae,
                    #     'last_ndcg_lin': last_ndcg_lin,
                    #     'last_ndcg_exp': last_ndcg_exp
                    # }
                    print(f"{model} & {confi} & {fc} & {min_mse:.4f} ({min_mse_epoch}) & {last_mse:.4f} & {min_mae:.4f} ({min_mae_epoch}) & {last_mae:.4f} & {max_ndcg_lin:.4f} ({max_ndcg_lin_epoch}) & {last_ndcg_lin:.4f} & {max_ndcg_exp:.4f} ({max_ndcg_exp_epoch}) & {last_ndcg_exp:.4f}\\\\")
        print("\\bottomrule")
        print("\\end{tabular}")
        print("\\end{table*}")
        print("\\end{landscape}\n")

print("############################################################################")
for bias in biases:
    for dataset in datasets:
        # print(f'Processing {dataset}')
        print("\\begin{landscape}")
        print("\\begin{table*}[]")
        print("\\centering")
        print("\\caption{Test ",f"{dataset.upper()}", f"bias = {bias}","}")
        print("\\scriptsize % Adjust the text size here")
        print("\\begin{tabular}{lllllllllllllll}")
        print("\\toprule")
        print("Model & Act & FC & MSE & MSE(l) & MSEn & MSEn(l) & MAE & MAE(l) & MAEn & MAEn(l) & nDCG(ll) & nDCG(l) & nDCG(e) & nDCG(el)\\\\")
        # print("Model & Act & FC & Min MSE & Last MSE & Min MSE neg & Last MSE neg & Min MAE  & Last MAE & Min MAE neg & Last MAE neg & Max nDCG (l) & Last nDCG (l) & Max nDCG (e) & Last nDCG (e)\\\\")
        for model in models:
            print("\\midrule")
            for confi in confi_score_functions:
                for fc in fc_layers:
                    result_path = osp.join(result_base_path, f'unc_{dataset}_{model}_confi_{confi}_fc_{fc}_bias_{bias}_dim_128', f'lr_0.01_wd_0.0_topk_True')
                    test_df = pd.read_csv(osp.join(result_path, 'test_metrics.csv'))
                    min_mse = test_df['MSE'].min()
                    min_mse_epoch = test_df[test_df['MSE'] == min_mse]['Epoch'].values[0]
                    min_mae = test_df['MAE'].min()
                    min_mae_epoch = test_df[test_df['MAE'] == min_mae]['Epoch'].values[0]
                    min_mse_with_neg = test_df['MSE_with_neg'].min()
                    min_mse_with_neg_epoch = test_df[test_df['MSE_with_neg'] == min_mse_with_neg]['Epoch'].values[0]
                    min_mae_with_neg = test_df['MAE_with_neg'].min()
                    min_mae_with_neg_epoch = test_df[test_df['MAE_with_neg'] == min_mae_with_neg]['Epoch'].values[0]
                    max_ndcg_lin = test_df['nDCG_lin'].max()
                    max_ndcg_lin_epoch = test_df[test_df['nDCG_lin'] == max_ndcg_lin]['Epoch'].values[0]
                    max_ndcg_exp = test_df['nDCG_exp'].max()
                    max_ndcg_exp_epoch = test_df[test_df['nDCG_exp'] == max_ndcg_exp]['Epoch'].values[0]
                    last_mse = test_df['MSE'].iloc[-1]
                    last_mae = test_df['MAE'].iloc[-1]
                    last_mae_with_neg = test_df['MAE_with_neg'].iloc[-1]
                    last_mse_with_neg = test_df['MSE_with_neg'].iloc[-1]
                    last_ndcg_lin = test_df['nDCG_lin'].iloc[-1]
                    last_ndcg_exp = test_df['nDCG_exp'].iloc[-1]
                    if dataset not in val_results:
                        val_results[dataset] = {}
                    if model not in val_results[dataset]:
                        val_results[dataset][model] = {}
                    if confi not in val_results[dataset][model]:
                        val_results[dataset][model][confi] = {}
                    # test_results[dataset][model][confi][fc] = {
                    #     'min_mse': min_mse,
                    #     'min_mse_epoch': min_mse_epoch,
                    #     'min_mae': min_mae,
                    #     'min_mae_epoch': min_mae_epoch,
                    #     'min_mse_with_neg': min_mse_with_neg,
                    #     'min_mse_with_neg_epoch': min_mse_with_neg_epoch,
                    #     'min_mae_with_neg': min_mae_with_neg,
                    #     'min_mae_with_neg_epoch': min_mae_with_neg_epoch,
                    #     'max_ndcg_lin': max_ndcg_lin,
                    #     'max_ndcg_lin_epoch': max_ndcg_lin_epoch,
                    #     'max_ndcg_exp': max_ndcg_exp,
                    #     'max_ndcg_exp_epoch': max_ndcg_exp_epoch,
                    #     'last_mse': last_mse,
                    #     'last_mae': last_mae,
                    #     'last_ndcg_lin': last_ndcg_lin,
                    #     'last_ndcg_exp': last_ndcg_exp
                    # }
                    # print(f'\t\t\tmin_mse: {min_mse} at epoch {min_mse_epoch}')
                    # print(f'\t\t\tmin_mae: {min_mae} at epoch {min_mae_epoch}')
                    # print(f'\t\t\tlast_mse: {last_mse}')
                    # print(f'\t\t\tlast_mae: {last_mae}')
                    # print(f'\t\t\tmax_ndcg_lin: {max_ndcg_lin} at epoch {max_ndcg_lin_epoch}')
                    # print(f'\t\t\tmax_ndcg_exp: {max_ndcg_exp} at epoch {max_ndcg_exp_epoch}')
                    print(f"{model} & {confi} & {fc} & {min_mse:.4f} ({min_mse_epoch}) & {last_mse:.4f} & {min_mse_with_neg:.4f} ({min_mse_with_neg_epoch}) & {last_mse_with_neg:.4f} & {min_mae:.4f} ({min_mae_epoch}) & {last_mae:.4f} & {min_mae_with_neg:.4f} ({min_mae_with_neg_epoch}) & {last_mae_with_neg:.4f} & {max_ndcg_lin:.4f} ({max_ndcg_lin_epoch}) & {last_ndcg_lin:.4f} & {max_ndcg_exp:.4f} ({max_ndcg_exp_epoch}) & {last_ndcg_exp:.4f}\\\\")
        print("\\bottomrule")
        print("\\end{tabular}")
        print("\\end{table*}")
        print("\\end{landscape}\n")

\begin{landscape}
\begin{table*}[]
\centering
\caption{Validation  CN15K bias = False }
\begin{tabular}{lllllllllll}
\toprule
Model & Act & FC & MSE & MSE(l) & MAE & MAE(l) & nDCG(ll) & nDCG(l) & nDCG(e) & nDCG(el)\\
\midrule
transe & logi & l1 & 0.2399 (1) & 0.2485 & 0.5652 (54) & 0.5676 & 0.5802 (4) & 0.5323 & 0.5727 (4) & 0.5233\\
transe & logi & l3 & 0.2399 (61) & 0.2507 & 0.5586 (61) & 0.5701 & 0.5876 (3) & 0.5336 & 0.5803 (3) & 0.5246\\
transe & logi & none & 0.2380 (74) & 0.2380 & 0.5555 (74) & 0.5557 & 0.6370 (2) & 0.5372 & 0.6296 (2) & 0.5284\\
transe & rect & l1 & 0.2110 (43) & 0.2180 & 0.5344 (43) & 0.5433 & 0.5107 (16) & 0.4958 & 0.5036 (16) & 0.4882\\
transe & rect & l3 & 0.2056 (76) & 0.2203 & 0.5270 (76) & 0.5464 & 0.5098 (14) & 0.4941 & 0.5027 (14) & 0.4864\\
transe & rect & none & 0.7740 (1) & 0.7740 & 1.0721 (1) & 1.0721 & 0.3316 (1) & 0.3316 & 0.3204 (1) & 0.3204\\
\midrule
distmult & logi & l1 & 0.1939 (1) & 0.3752 & 0.4679 (2) & 0.6468 & 0.8490 (99) & 0.8476 & 0.85

In [9]:
# checkpoint = torch.load('/home/mou/Projects/UKGE-FL/results/unc_cn15k_distmult_confi_logi_fc_l1_bias_True_dim_128/lr_0.01_wd_0.0_topk_True/best_model_ndcg_lin.pth')
# state_dict = checkpoint['state_dict']

# print(state_dict.keys())
# print(state_dict['fc.bias'])
# print(state_dict['fc.weight'])

## Latex: fc=l1, bias=True

In [12]:
biases = [False, True]
lrs = [0.01, 0.005, 0.001]
hidden_dims = [128, 256, 512]
models = ['transe', 'distmult', 'complex', 'rotate']
datasets = ['cn15k', 'nl27k', 'ppi5k']
confi_score_functions = ['logi', 'rect']
fc_layers = ['l1', 'l3', 'none']
val_results = {}
result_base_path = '/home/mou/Projects/UKGE-FL/results'

In [14]:
val_results = {}
result_base_path = '/home/mou/Projects/UKGE-FL/results'

for lr in lrs:
    for dataset in datasets:
        # print(f'Processing {dataset}')
        print("\\begin{landscape}")
        print("\\begin{table*}[]")
        print("\\centering")
        print("\\caption{Validation ", f"{dataset.upper()}", f"lr = {lr})", "}")
        print("\\begin{tabular}{lllllllllll}")
        print("\\toprule")
        print("Model & Act & Dim & MSE & MSE(l) & MAE & MAE(l) & nDCG(ll) & nDCG(l) & nDCG(e) & nDCG(el)\\\\")
        for model in models:
            print("\\midrule")
            for dim in hidden_dims:
                result_path = osp.join(result_base_path, f'unc_{dataset}_{model}_confi_{confi}_fc_l1_bias_True_dim_{dim}', f'lr_{lr}_wd_0.0_topk_True')
                val_df = pd.read_csv(osp.join(result_path, 'val_metrics.csv'))
                min_mse = val_df['MSE'].min()
                min_mse_epoch = val_df[val_df['MSE'] == min_mse]['Epoch'].values[0]
                min_mae = val_df['MAE'].min()
                min_mae_epoch = val_df[val_df['MAE'] == min_mae]['Epoch'].values[0]
                max_ndcg_lin = val_df['nDCG_lin'].max()
                max_ndcg_lin_epoch = val_df[val_df['nDCG_lin'] == max_ndcg_lin]['Epoch'].values[0]
                max_ndcg_exp = val_df['nDCG_exp'].max()
                max_ndcg_exp_epoch = val_df[val_df['nDCG_exp'] == max_ndcg_exp]['Epoch'].values[0]
                last_mse = val_df['MSE'].iloc[-1]
                last_mae = val_df['MAE'].iloc[-1]
                last_ndcg_lin = val_df['nDCG_lin'].iloc[-1]
                last_ndcg_exp = val_df['nDCG_exp'].iloc[-1]
                # if dataset not in val_results:
                #     val_results[dataset] = {}
                # if model not in val_results[dataset]:
                #     val_results[dataset][model] = {}
                # if confi not in val_results[dataset][model]:
                #     val_results[dataset][model][confi] = {}
                # val_results[dataset][model][confi]['lr'][True][dim][lr] = {
                #     'min_mse': min_mse,
                #     'min_mse_epoch': min_mse_epoch,
                #     'min_mae': min_mae,
                #     'min_mae_epoch': min_mae_epoch,
                #     'max_ndcg_lin': max_ndcg_lin,
                #     'max_ndcg_lin_epoch': max_ndcg_lin_epoch,
                #     'max_ndcg_exp': max_ndcg_exp,
                #     'max_ndcg_exp_epoch': max_ndcg_exp_epoch,
                #     'last_mse': last_mse,
                #     'last_mae': last_mae,
                #     'last_ndcg_lin': last_ndcg_lin,
                #     'last_ndcg_exp': last_ndcg_exp
                # }
                print(f"{model} & {confi} & {dim} & {min_mse:.4f} ({min_mse_epoch}) & {last_mse:.4f} & {min_mae:.4f} ({min_mae_epoch}) & {last_mae:.4f} & {max_ndcg_lin:.4f} ({max_ndcg_lin_epoch}) & {last_ndcg_lin:.4f} & {max_ndcg_exp:.4f} ({max_ndcg_exp_epoch}) & {last_ndcg_exp:.4f}\\\\")
        print("\\bottomrule")
        print("\\end{tabular}")
        print("\\end{table*}")
        print("\\end{landscape}\n")

print("############################################################################")
for lr in lrs:
    for dataset in datasets:
        # print(f'Processing {dataset}')
        print("\\begin{landscape}")
        print("\\begin{table*}[]")
        print("\\centering")
        print("\\caption{Test ", f"{dataset.upper()}", f"lr = {lr})", "}")
        print("\\scriptsize % Adjust the text size here")
        print("\\begin{tabular}{lllllllllllllll}")
        print("\\toprule")
        print("Model & Act & Dim & MSE & MSE(l) & MSEn & MSEn(l) & MAE & MAE(l) & MAEn & MAEn(l) & nDCG(ll) & nDCG(l) & nDCG(e) & nDCG(el)\\\\")
        # print("Model & Act & FC & Min MSE & Last MSE & Min MSE neg & Last MSE neg & Min MAE  & Last MAE & Min MAE neg & Last MAE neg & Max nDCG (l) & Last nDCG (l) & Max nDCG (e) & Last nDCG (e)\\\\")
        for model in models:
            print("\\midrule")
            for confi in confi_score_functions:
                for dim in hidden_dims:
                    result_path = osp.join(result_base_path, f'unc_{dataset}_{model}_confi_{confi}_fc_l1_bias_True_dim_{dim}', f'lr_{lr}_wd_0.0_topk_True')
                    test_df = pd.read_csv(osp.join(result_path, 'test_metrics.csv'))
                    min_mse = test_df['MSE'].min()
                    min_mse_epoch = test_df[test_df['MSE'] == min_mse]['Epoch'].values[0]
                    min_mae = test_df['MAE'].min()
                    min_mae_epoch = test_df[test_df['MAE'] == min_mae]['Epoch'].values[0]
                    min_mse_with_neg = test_df['MSE_with_neg'].min()
                    min_mse_with_neg_epoch = test_df[test_df['MSE_with_neg'] == min_mse_with_neg]['Epoch'].values[0]
                    min_mae_with_neg = test_df['MAE_with_neg'].min()
                    min_mae_with_neg_epoch = test_df[test_df['MAE_with_neg'] == min_mae_with_neg]['Epoch'].values[0]
                    max_ndcg_lin = test_df['nDCG_lin'].max()
                    max_ndcg_lin_epoch = test_df[test_df['nDCG_lin'] == max_ndcg_lin]['Epoch'].values[0]
                    max_ndcg_exp = test_df['nDCG_exp'].max()
                    max_ndcg_exp_epoch = test_df[test_df['nDCG_exp'] == max_ndcg_exp]['Epoch'].values[0]
                    last_mse = test_df['MSE'].iloc[-1]
                    last_mae = test_df['MAE'].iloc[-1]
                    last_mae_with_neg = test_df['MAE_with_neg'].iloc[-1]
                    last_mse_with_neg = test_df['MSE_with_neg'].iloc[-1]
                    last_ndcg_lin = test_df['nDCG_lin'].iloc[-1]
                    last_ndcg_exp = test_df['nDCG_exp'].iloc[-1]
                    # if dataset not in test_results:
                    #     test_results[dataset] = {}
                    # if model not in test_results[dataset]:
                    #     val_results[dataset][model] = {}
                    # if confi not in test_results[dataset][model]:
                    #     val_results[dataset][model][confi] = {}
                    # test_results[dataset][model][confi]['lr'][True][dim][lr] = {
                    #     'min_mse': min_mse,
                    #     'min_mse_epoch': min_mse_epoch,
                    #     'min_mae': min_mae,
                    #     'min_mae_epoch': min_mae_epoch,
                    #     'min_mse_with_neg': min_mse_with_neg,
                    #     'min_mse_with_neg_epoch': min_mse_with_neg_epoch,
                    #     'min_mae_with_neg': min_mae_with_neg,
                    #     'min_mae_with_neg_epoch': min_mae_with_neg_epoch,
                    #     'max_ndcg_lin': max_ndcg_lin,
                    #     'max_ndcg_lin_epoch': max_ndcg_lin_epoch,
                    #     'max_ndcg_exp': max_ndcg_exp,
                    #     'max_ndcg_exp_epoch': max_ndcg_exp_epoch,
                    #     'last_mse': last_mse,
                    #     'last_mae': last_mae,
                    #     'last_ndcg_lin': last_ndcg_lin,
                    #     'last_ndcg_exp': last_ndcg_exp
                    # }
                    # print(f'\t\t\tmin_mse: {min_mse} at epoch {min_mse_epoch}')
                    # print(f'\t\t\tmin_mae: {min_mae} at epoch {min_mae_epoch}')
                    # print(f'\t\t\tlast_mse: {last_mse}')
                    # print(f'\t\t\tlast_mae: {last_mae}')
                    # print(f'\t\t\tmax_ndcg_lin: {max_ndcg_lin} at epoch {max_ndcg_lin_epoch}')
                    # print(f'\t\t\tmax_ndcg_exp: {max_ndcg_exp} at epoch {max_ndcg_exp_epoch}')
                    print(f"{model} & {confi} & {dim} & {min_mse:.4f} ({min_mse_epoch}) & {last_mse:.4f} & {min_mse_with_neg:.4f} ({min_mse_with_neg_epoch}) & {last_mse_with_neg:.4f} & {min_mae:.4f} ({min_mae_epoch}) & {last_mae:.4f} & {min_mae_with_neg:.4f} ({min_mae_with_neg_epoch}) & {last_mae_with_neg:.4f} & {max_ndcg_lin:.4f} ({max_ndcg_lin_epoch}) & {last_ndcg_lin:.4f} & {max_ndcg_exp:.4f} ({max_ndcg_exp_epoch}) & {last_ndcg_exp:.4f}\\\\")
        print("\\bottomrule")
        print("\\end{tabular}")
        print("\\end{table*}")
        print("\\end{landscape}\n")

\begin{landscape}
\begin{table*}[]
\centering
\caption{Validation  CN15K lr = 0.01)
\begin{tabular}{lllllllllll}
\toprule
Model & Act & Dim & MSE & MSE(l) & MAE & MAE(l) & nDCG(ll) & nDCG(l) & nDCG(e) & nDCG(el)\\
\midrule
transe & rect & 128 & 0.1754 (4) & 0.3501 & 0.4466 (6) & 0.6288 & 0.8009 (94) & 0.7966 & 0.7992 (94) & 0.7947\\
transe & rect & 256 & 0.2887 (1) & 0.2887 & 0.5868 (1) & 0.5868 & 0.3316 (1) & 0.3316 & 0.3204 (1) & 0.3204\\
transe & rect & 512 & 0.7740 (1) & 0.7740 & 1.0721 (1) & 1.0721 & 0.3316 (1) & 0.3316 & 0.3204 (1) & 0.3204\\
\midrule
distmult & rect & 128 & 0.1916 (1) & 0.3847 & 0.4664 (1) & 0.6564 & 0.8439 (93) & 0.8419 & 0.8455 (93) & 0.8432\\
distmult & rect & 256 & 0.1915 (1) & 0.3834 & 0.4636 (1) & 0.6596 & 0.8779 (100) & 0.8779 & 0.8788 (98) & 0.8787\\
distmult & rect & 512 & 0.7740 (1) & 0.7740 & 1.0721 (1) & 1.0721 & 0.3316 (1) & 0.3316 & 0.3204 (1) & 0.3204\\
\midrule
complex & rect & 128 & 0.1908 (1) & 0.4050 & 0.4652 (1) & 0.6916 & 0.9322 (100) & 0.93

In [1]:
import os
import os.path as osp
import torch

from ukge.datasets import KGTripleDataset
from ukge.models import DistMult

base_path = '/home/mou/Projects/UKGE-FL/results'
dataset = 'cn15k'
model = 'distmult'
exp_path = osp.join(f'psl_{dataset}_{model}_confi_logi_fc_l1_bias_True_dim_256', f'lr_0.01_wd_0.0_topk_True')
checkpoint = torch.load(osp.join(base_path, exp_path, 'best_model_ndcg_lin.pth'))
state_dict = checkpoint['state_dict']


test_dataset = KGTripleDataset(dataset=dataset, split='test', topk=True, k=200)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=4)
batch_data = next(iter(test_dataloader))

model = DistMult(num_nodes=test_dataset.num_cons(), num_relations=test_dataset.num_rels(), hidden_channels=256, bias=True).to('cuda')

In [2]:
for name, param in state_dict.items():
    print(name)
    print(param)
    print('#####################')

node_emb.weight
tensor([[ 0.2363,  0.1371, -1.2904,  ..., -0.5430, -0.0685, -0.6315],
        [-0.1733,  1.0557,  0.5243,  ...,  0.1953,  3.2021,  2.5010],
        [ 1.5751, -1.2010,  1.1324,  ...,  1.0853,  0.2869, -0.0280],
        ...,
        [ 0.8059, -0.5701, -0.6178,  ...,  0.5701,  0.7011,  1.6636],
        [-0.6600,  0.7024, -0.3744,  ...,  0.9745,  0.0657,  0.7779],
        [-0.6052,  0.4213,  0.1795,  ..., -0.6679, -0.2127, -0.6621]],
       device='cuda:0')
#####################
rel_emb.weight
tensor([[-1.1798, -1.0080, -0.5911,  ..., -0.6298, -1.3480, -1.0629],
        [ 0.7889, -1.5040, -0.1380,  ..., -0.1281, -0.7851, -1.2280],
        [-1.4914, -1.3987, -1.3219,  ..., -0.2236, -1.2893, -1.3913],
        ...,
        [ 0.8952, -0.5014,  1.5818,  ...,  0.8880, -2.5834, -1.3490],
        [-0.5935, -0.6023, -0.8025,  ..., -0.2930, -1.5080,  0.0331],
        [ 0.2116,  0.4306, -0.2969,  ...,  0.9877, -0.1362, -0.5913]],
       device='cuda:0')
#####################
fc.weight

In [14]:

hrt, s = batch_data
hrt = hrt.long().to('cuda')
p_scores = model.get_plausibility_score(hrt[:, 0], hrt[:, 1], hrt[:, 2])
c_scores = model.get_confidence_score(hrt[:, 0], hrt[:, 1], hrt[:, 2])
hrt, pred_p, pred_s= hrt.detach().cpu().numpy(), p_scores.detach().cpu().numpy(), c_scores.detach().cpu().numpy()
print(s)
print(p_scores)
print(c_scores)

tensor([0.7093, 0.7093, 0.1129, 0.2570, 0.8927, 0.5794, 0.7093, 0.8927, 0.7093,
        0.7093], dtype=torch.float64)
tensor([ 1.1548e-04,  1.9559e-04, -6.7839e-05, -9.6022e-05, -1.5021e-04,
         2.7664e-04, -1.3525e-04, -4.3159e-04, -1.6303e-04,  1.6849e-04],
       device='cuda:0', grad_fn=<SumBackward1>)
tensor([0.3589, 0.3589, 0.3589, 0.3589, 0.3589, 0.3589, 0.3589, 0.3588, 0.3589,
        0.3589], device='cuda:0', grad_fn=<SigmoidBackward0>)
